In [1]:
from bs4 import BeautifulSoup
from requests import get
import cfscrape
import re

In [2]:
scraper = cfscrape.create_scraper()  
proxies={
    'http':'http://35.169.156.54:3128',
    'https':'https://35.169.156.54:3128',
}
t_id=[] #each transaction id 
t_timestamp=[] #timestamp for each transaction


t_amount=[] #each transaction prices

t_block=[] #the block each transaction belongs to
t_blockset=set() #counts the number of different blocks number to make sure we webscrape the most recent 100 transactions

for page in range(1,301): #each block at most contains 300 transactions, and each page contains 100 transactions, so we need to go over at most 300 pages 
    url= 'http://etherscan.io/txs?ps=100&p='+str(page)
    htmltext = scraper.get(url)

    if htmltext.status_code == 200:
        soup_object = BeautifulSoup(htmltext.content, 'html.parser')
   
    flag=0
    for p in soup_object.tbody.find_all('a'):
        p2=p.get('href')
        if p2:
            if 'block' in p2:
                t_blockset.add(p2.replace('/block/',''))
                if len(t_blockset)==101:
                    flag=1
                    break
                
                t_block.append(p2.replace('/block/','')) #include all blocks' number into t_block

        p1=p.get('class')
        
        if p1:
            if 'myFnExpandBox_searchVal' in p1:
                id_temp=p.get('href')
                t_id.append(id_temp.replace('/tx/','')) #include all transactions' id into t_id

    if flag==1:
        break

In [3]:
t_urlid=[] #each transaction id's website
t_source=[] #the source address of each transaction
t_destination=[] #the destination address of each transaction
for x in t_id:
    url_id= 'https://etherscan.io/tx/'+str(x)
    t_urlid.append(url_id)
    htmltextid = scraper.get(url_id)
    if htmltextid.status_code == 200:
        soup_objectid = BeautifulSoup(htmltextid.content, 'html.parser')
    
    for t in soup_objectid.find_all('a'):
        t1=t.get('id')
        if t1:
            if t1=='addressCopy':
                t1_addresstem=t.get('href')
                t_source.append(t1_addresstem.replace('/address/','')) #include all transactions' address into t_source
            elif t1=='contractCopy':
                t1_destinationtemp=t.get('href')
                t_destination.append(t1_destinationtemp.replace('/address/','')) #include all transactions' destination into t_destination

In [4]:
# get transactions' timestamp and transanction amount
t_urldetails=[] #store each transactions' id corresponding websites 
for i in t_id:
    url_details= 'https://etherscan.io/tx/'+ str(i)
    t_urldetails.append(url_details)
    htmltextd = scraper.get(url_details)
    if htmltextd.status_code == 200:
        soup_objectd = BeautifulSoup(htmltextd.content, 'html.parser')

    for t in soup_objectd.find_all('span'):
        t1=t.get('id')

        if t1:
            if 'ContentPlaceHolder1_spanValue' in t1:
                amount_initial=t.find(text=lambda text: text and "$" in text)
                amount_temp=amount_initial.replace(' (','')
                amount=amount_temp.replace(')','')
                t_amount.append(amount) #include all transactions' amount into t_amount
            elif 'clock' in t1:
                t_temp=t.nextSibling
                t_tempNext=t_temp.nextSibling
                time_temp=re.sub(r'.*ago ','',t_tempNext)
                time_temp1=time_temp.replace('\n','')
                time=time_temp1.strip('()')
                t_timestamp.append(time) #include all transactions' timestamp into t_timestamp

In [5]:
#get transactions' mining pools
t_blockdetails=[]
t_pool=[]
count=0
for i in t_block:
    url_blockdetails='https://etherscan.io/block/'+str(i)
    t_blockdetails.append(url_blockdetails)
    htmltextd = scraper.get(url_blockdetails)
    if htmltextd.status_code == 200:
        soup_objectd = BeautifulSoup(htmltextd.content, 'html.parser')
    
    for h in soup_objectd.find_all('b'):
        h_text=h.getText()
        t_pool.append(h_text) #include all transactions' mining pools into t_pool
        break

In [6]:
whole_listtemp=zip(t_id,t_block,t_timestamp,t_source,t_destination,t_amount,t_pool)
whole_list1=list(whole_listtemp)
whole_list=sorted(whole_list1, key=lambda x: x[1], reverse=True) #sort transactions based on blocks from the latest to the oldest

In [7]:
import csv

with open('ethereum_data1.csv', 'w') as fp:
    writer = csv.writer(fp, delimiter = ',')
    writer.writerow(["transaction_ID","block" ,"transaction_timestamp", "source_address","destination_address","transaction_amount","mining_pool"])
    for row in whole_list:
        writer.writerow(row)